In [2]:
import pandas as pd

orders = pd.read_parquet("Dateien/orders.parquet")  
tips_public = pd.read_csv("Dateien/tips_public.csv", index_col=0)  
tip_testdaten_template = pd.read_csv("Dateien/tip_testdaten_template.csv")  
order_products_denormalized = pd.read_csv("Dateien/order_products_denormalized.csv")  


In [3]:
max_lens = {col: order_products_denormalized[col].astype(str).map(len).max() for col in order_products_denormalized.columns}
max_lens 

{'order_id': np.int64(7),
 'product_id': np.int64(5),
 'add_to_cart_order': np.int64(3),
 'product_name': np.int64(159),
 'aisle_id': np.int64(3),
 'department_id': np.int64(2),
 'department': np.int64(15),
 'aisle': np.int64(29)}

In [4]:
user_tip = orders.merge(tips_public, how="inner", on="order_id")
user_tip = user_tip.sort_values(["user_id","order_date"])

# Previous x tips
for i in range(1, 10):
    user_tip[f"tip_{i}"] = user_tip.groupby("user_id")["tip"].shift(i)#.astype("bool")


# tip_columns = [col for col in user_tip.columns if 'tip' in col.lower()]
# user_tip = user_tip[tip_columns]
# user_tip = user_tip.dropna() 

In [7]:
d = pd.DataFrame(orders.groupby("user_id")["order_id"].count()) \
    .merge(orders[["user_id"]], how="inner", on="user_id") \
    .drop_duplicates()[["user_id", "order_id"]] \
    .sort_values("user_id") \
    .set_index("user_id") \
    .rename(columns={"order_id": "n_orders"}) \
    .join(user_tip[["user_id", "tip"]].groupby("user_id")["tip"].sum().reset_index().set_index("user_id").rename(columns={"tip": "n_tips"})) \
    .join(user_tip[["user_id", "tip"]].groupby("user_id")["tip"].mean().reset_index().set_index("user_id").rename(columns={"tip": "mean_tip"})) \
    .join(user_tip[["user_id", "tip"]].groupby("user_id")["tip"].std().reset_index().set_index("user_id").rename(columns={"tip": "std_tip"})) \
    .join(user_tip[["user_id", "tip"]].groupby("user_id")["tip"].min().reset_index().set_index("user_id").rename(columns={"tip": "min_tip"})) \
    .join(user_tip[["user_id", "tip"]].groupby("user_id")["tip"].max().reset_index().set_index("user_id").rename(columns={"tip": "max_tip"}))   # does at least one tip exist (max=True)  # does a user tip only (min=True) 


d = d.reset_index()

d


,user_id,n_orders,n_tips,mean_tip,std_tip,min_tip,max_tip
0,3,13,10,0.833333,0.389249,False,True
1,5,5,2,0.500000,0.577350,False,True
2,6,4,0,0.000000,0.000000,False,False
3,13,13,8,0.666667,0.492366,False,True
4,15,23,9,0.409091,0.503236,False,True
...,...,...,...,...,...,...,...
62027,206187,35,32,0.941176,0.238833,False,True
62028,206189,7,1,0.166667,0.408248,False,True
62029,206204,5,1,0.250000,0.500000,False,True
62030,206206,68,15,0.223881,0.419989,False,True
